### Agent

An AI agent is an LLM-powered system that can autonomously think, decide and take actions using external tools or APIs to achieve a goal.

### Tool

A tool is just a python function (API) that is packaged in a way that the LLM can understand and call when needed.

In [ ]:
import os
import sys

from dotenv import load_dotenv
load_dotenv()

os.environ['HF_HOME']="/Users/nikhilsharma/Desktop/hf-cache"
# os.environ['HF_HOME']="/Users/nikhil20.sharma/Desktop/hf-cache"

# Print all environment variables loaded from .env
print("Loaded Environment Variables:")
for key, value in os.environ.items():
    if key in ['OPENAI_API_KEY', 'LANGSMITH_AIP_KEY', 'HUGGINGFACE_TOKEN']:
        # Mask sensitive values for security
        masked_value = value[:8] + "..." + value[-4:] if value else value
        print(f"- {key}: {masked_value}")

Loaded Environment Variables:
- OPENAI_API_KEY: sk-proj-...4bIA
- HUGGINGFACE_TOKEN: hf_kVKaN...KHxS


### Tool Calling

**Tool Calling** is the process where the LLM decides, during a conversation or task, that it needs to use a specific tool (function) - and generates a structured output with:
- the **name of the tool**
- and the **arguments** to call it with

**Note:** 
- The LLM does not actually run the tool.
- It just suggests the tool and the input arguments.
- The actual execution is handled by the LangChain or you.

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [40]:
# TOOL CREATION

In [55]:
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [56]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [57]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [35]:
# TOOL BINDING

In [37]:
llm = ChatOpenAI()

In [38]:
llm.invoke('hi')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BZIKMjVtI3rw4zK7ngNfOpCHgGQSM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3a92d24f-7109-43f2-a80a-1c11c60ee180-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [39]:
llm_with_tools = llm.bind_tools([multiply])

In [41]:
# TOOL CALLING

In [45]:
result = llm_with_tools.invoke("Can you tell me what is 5 times 10?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2oQVkyzh3zrqoBnMbRQasPBA', 'function': {'arguments': '{"a":5,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 66, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BZLRCXstZclsri1ieLi0gNiJHFnkZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3c269bc0-2df5-4630-adfd-f4b9c20c2da0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 10}, 'id': 'call_2oQVkyzh3zrqoBnMbRQasPBA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 66, 'output_tokens': 17, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 

In [47]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 5, 'b': 10},
 'id': 'call_2oQVkyzh3zrqoBnMbRQasPBA',
 'type': 'tool_call'}

In [44]:
# TOOL EXECUTION

In [50]:
multiply.invoke(result.tool_calls[0]['args'])

50

In [51]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='50', name='multiply', tool_call_id='call_2oQVkyzh3zrqoBnMbRQasPBA')

### END TO END - CURRENCY CONVERSION TOOL

In [58]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [60]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [61]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [62]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1747958401,
 'time_last_update_utc': 'Fri, 23 May 2025 00:00:01 +0000',
 'time_next_update_unix': 1748044801,
 'time_next_update_utc': 'Sat, 24 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.9969}

In [ ]:
convert.invoke({'base_currency_value': 10, 'conversion_rate': 85.16})

851.5999999999999

In [64]:
# TOOL BINDING

In [65]:
llm = ChatOpenAI()

In [66]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [67]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [68]:
ai_message = llm_with_tools.invoke(messages)

In [69]:
messages.append(ai_message)

In [70]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_JwpmkqCuAmQn12slUHW21QUU',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_8squgokRJHbtGLpG5dkGM6ZX',
  'type': 'tool_call'}]

In [71]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [72]:
for message in messages:
    print(message, end='\n\n')

content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd' additional_kwargs={} response_metadata={}

content='' additional_kwargs={'tool_calls': [{'id': 'call_JwpmkqCuAmQn12slUHW21QUU', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_8squgokRJHbtGLpG5dkGM6ZX', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 123, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BaQ0LdNsASvlXs4be2wCOgKbwgwY5', 'service_tier': 'default', 'finish_reason': 'tool_c

In [73]:
response = llm_with_tools.invoke(messages)
response.content

'The conversion factor between INR (Indian Rupee) and USD (US Dollar) is 0.01163.\n\nBased on this conversion factor, converting 10 INR to USD would be approximately 0.1163 USD.'

### Built-in Tools

##### DuckDuckGo Search

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('News on Bengaluru Rain')

print(results)

Bengaluru Rains: Heavy rainfall took Bengaluru by storm on Sunday, with several areas reporting severe waterlogging. As per Karnataka Weatherman, the deluge last night was Bengaluru's heaviest of the year yet. The India Meteorological Department (IMD) has issued a heavy rain alert in the city., Bengaluru News News - Times Now Karnataka Weather: Bengaluru sees severe flooding, traffic chaos, and casualties as IMD issues orange alert. Stay updated with our live coverage. Bengaluru rain LIVE updates: Follow The Hindu's live updates as heavy rain lash Karnataka's capital Bengaluru on May 20, 2025. Bengaluru weather live updates: Incessant rainfall has wreaked havoc in the capital of Karnataka as torrential downpour led to massive waterlogging and flooding on the city roads, posing severe ... Bangalore Rain News Today, Weather Updates Live: Bengaluru received record rainfall in the last 24 hours, logging 105.5 mm of rainfall during the 24 hours ending 8.30 am on May 19.


##### Shell Tool

In [4]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
1-learn-rag-components.ipynb
2-learn-agent-components.ipynb
3.prompt_generator.py
DB
README.md
chat_history.txt
data
deeplearning.ai
langchain-nlp-course.ipynb
notes.txt
programming_practice.ipynb
project-email-sql-with-natural-language
project-pdf-conversation
project-stt-whisper
project-youtube-video-chat
prompt_template.json
requirements.txt
transformer-explained-2.ipynb
transformer-explained.ipynb
transformers_code.ipynb
vector-query-with-metadata-approach.md
word2vec_cbow.ipynb
word2vec_skipgram.ipynb



/Users/nikhilsharma/miniconda3/envs/lang/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom Tools

In [5]:
from langchain_core.tools import tool

In [6]:
# Step 1 - create a function
# Step 2 - add type hints
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [8]:
result = multiply.invoke({"a":3, "b":5})
result

15

In [9]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [11]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


In [10]:
print(shell_tool.name)
print(shell_tool.description)
print(shell_tool.args)

terminal
Run shell commands on this MacOS machine.
{'commands': {'anyOf': [{'type': 'string'}, {'items': {'type': 'string'}, 'type': 'array'}], 'description': 'List of shell commands to run. Deserialized using json.loads', 'title': 'Commands'}}


In [14]:
multiply.args_schema.model_json_schema()

{'description': 'Multiply two numbers',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

In [15]:
search_tool.args_schema.model_json_schema()

{'description': 'Input for the DuckDuckGo search tool.',
 'properties': {'query': {'description': 'search query to look up',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'DDGInput',
 'type': 'object'}

In [16]:
shell_tool.args_schema.model_json_schema()

{'description': 'Commands for the Bash Shell tool.',
 'properties': {'commands': {'anyOf': [{'type': 'string'},
    {'items': {'type': 'string'}, 'type': 'array'}],
   'description': 'List of shell commands to run. Deserialized using json.loads',
   'title': 'Commands'}},
 'required': ['commands'],
 'title': 'ShellInput',
 'type': 'object'}

### Using StructuredTool

In [17]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [18]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [19]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [20]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [21]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Using BaseTool Class

In [22]:
from langchain.tools import BaseTool
from typing import Type

In [23]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [24]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [25]:
multiply_tool = MultiplyTool()

In [26]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Toolkit

A toolkit is just a collection (bundle) of related tools that serve a common purpose - packaged together for convienence and reusability.

In [27]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [28]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]

In [29]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
